In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.webgui import Draw as Drawgeo
import matplotlib.pyplot as plt
import numpy as np

importing NGSolve-6.2.2301


Consider the following. Find $u$ satisfying
\begin{equation}
-\nabla u - \omega^2 u = f \text{ in } \mathbb{R}^2,
\end{equation}
with $u$ satisfying the Sommerfeld radiation condition
\begin{equation}
\lim_{r\to\infty} r^{1/2} \left(\frac{\partial u}{\partial r} - i\omega u\right) = 0,
\end{equation}
where $r$ is the radial coordinate from the origin.

In [2]:
outer = Circle((0,0), 2).Face()
outer.edges.name = 'outerbnd'
inner = Circle((0,0), 1).Face()
inner.edges.name = 'innerbnd'
inner.faces.name ='inner'
pmlregion = outer - inner
pmlregion.faces.name = 'pmlregion'
geo = OCCGeometry(Glue([inner, pmlregion]), dim=2)

mesh = Mesh(geo.GenerateMesh (maxh=0.1))
mesh.Curve(3)

inout_dict = {'inner': 1, 'pmlregion':0}
inout = CoefficientFunction([inout_dict[mat] for mat in mesh.GetMaterials()])

We set $f$ to be almost 0 everywhere except a small region away from the boundary,

In [3]:
f = exp(-20**2*((x-0.3)*(x-0.3)+y*y))
Draw(f, mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

Setting PML in pmlregion

In [4]:
mesh.SetPML(pml.Radial(rad=1,alpha=1j,origin=(0,0)), "pmlregion")

Solving H1 with $\omega=10$ using a direct inverse

In [5]:
fes = H1(mesh, order=4, complex=True)
u = fes.TrialFunction()
v = fes.TestFunction()

omega = 10

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx - omega**2*u*v*dx
a += -1j*omega*u*v*ds("outerbnd")
a.Assemble()

b = LinearForm(f * v * dx).Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse() * b.vec
Draw(gfu);



WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [6]:
Integrate(inout * gfu, mesh)

(-3.4890070419615374e-06-1.3102267046576246e-05j)

Trying again with my own implementation.

We want the field to decay exponentially to (near) 0 inside the PML. To do this, we update the material properties, $A$ inside the PML, i.e. rewritting the weak form of the problem as: find $u$ such that
\begin{equation}
\int_\Omega A^{-1} \nabla u \cdot \nabla v \, \text{d}\Omega - \int_\Omega A  \omega^2 u v \text{d}\Omega = \int_\Omega f v \text{d}\Omega
\end{equation}


In [7]:
outer = Circle((0,0), 2).Face()
outer.edges.name = 'outerbnd'
inner = Circle((0,0), 1).Face()
inner.edges.name = 'innerbnd'
inner.faces.name ='inner'
pmlregion = outer - inner
pmlregion.faces.name = 'pmlregion'
geo = OCCGeometry(Glue([inner, pmlregion]), dim=2)

mesh = Mesh(geo.GenerateMesh (maxh=0.1))
mesh.Curve(3)


fes = H1(mesh, order=4, complex=True, dirichlet='outerbnd')
u = fes.TrialFunction()
v = fes.TestFunction()
omega = 10

r = (x**2 + y**2)**(0.5)
r_inner = 1
r_outer = 2
alpha  = 20
sigma = 6 * ((r - r_inner) / (r_outer - r_inner)) ** 2
gamma = 1 + (1j *alpha * sigma)

amp_dict = {'inner': 1, 'pmlregion': gamma}

A = CoefficientFunction([amp_dict[mat] for mat in mesh.GetMaterials()])

a = BilinearForm(fes)
a += A**-1 *grad(u)*grad(v) * dx - A * omega**2*u*v * dx
a += A * -1j*omega*u*v * ds('outerbnd')
a.Assemble()

#f = exp(-20**2*((x+gamma-0.3)*(x+gamma-0.3)+y*gamma*y*gamma))

b = LinearForm(fes)
b += f * v * dx
b.Assemble()


gfu = GridFunction(fes)
gfu.Set(0, BND)
gfu.vec.data = a.mat.Inverse() * b.vec


Drawing the imaginary part of the weighting, $A$. Only imaginary part exists inside the pml.

In [8]:
Draw(A.imag, mesh, 'A')
Draw(A.real, mesh, 'A')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Similarly, the solution field, $u$, decays towards the outer boundary.

In [9]:
Draw(gfu)#, min=-0.00096, max=0.0018);


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [10]:
Integrate(inout * gfu, mesh)

(-3.4338226836368e-06-1.2647816710175955e-05j)

## Robust PML
In this third example I implement a PML following the implementation proposed by Michler $\textit{et al}$

Michler, C. and Demkowlcz, L. and Kurtz, J. and Pardo, D., "{Improving the performance of perfectly matched layers by means of hp-adaptivity}", Numer. Meth. PDEs. 23(4), pg. 832--858, 2007

we use the transformation $x_j \rightarrow x_j - \mathrm{i}b_j(x) = z_j(x_j)$ with $\frac{\partial}{\partial x_j} \rightarrow \frac{\partial}{\partial z_j(x_j)} = \frac{1}{z_j\prime(x_j)}\frac{\partial}{\partial x_j}$. For $z_j(x_j) = \mathrm{i}b_j(x_j)$, $z\prime_j =1 - \mathrm{i}\left(\frac{\partial b_j(x_j)}{\partial x_j} \right)$.

The updated differential equation is therefore $\frac{\partial}{\partial x_j} \rightarrow \frac{1}{1-\mathrm{i}\left(\frac{\partial b_j(x_j)}{\partial x_j} \right)}\frac{\partial}{\partial x_j}$.




In [11]:

wp = WorkPlane()
wp.MoveTo(-0.5,-0.5)
inner = wp.Rectangle(1,1).Face()
wp.MoveTo(-1,-1)
outer = wp.Rectangle(2,2).Face()


outer.edges.name = 'outerbnd'
inner.edges.name = 'innerbnd'
inner.faces.name ='inner'
pmlregion = outer - inner
pmlregion.faces.name = 'pmlregion'
geo = OCCGeometry(Glue([inner, pmlregion]), dim=2)

Drawgeo(Glue([inner, pmlregion]))

mesh = Mesh(geo.GenerateMesh (maxh=0.1))
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Choosing
\begin{equation}
z_j = \begin{cases}
x_j - \mathrm{i}\left( \frac{|x_j| - d}{0.5} \right)x_j &\text{ for } |x_j| > d \\
x_j &\text{ otherwise}
\end{cases}
\end{equation}
inside the PML.

The updated weak form is 
\begin{equation}

\end{equation}

In [12]:
def absval(x):
    return sqrt(x**2)

In [13]:
d = 2
z_x = IfPos(absval(x) - 0.5, x - (1j * ( absval(x) - d)/0.5)**1 * x, x) # returns z_j if |x|>0.5 else returns x
z_y = IfPos(absval(y) - 0.5, y - (1j * ( absval(y) - d)/0.5)**1 * y, y) # returns z_j if |y|>0.5 else returns y

Draw(z_x.imag, mesh)
Draw(z_y.imag, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [14]:
dzx = z_x.Diff(x)
dzy = z_y.Diff(y)

Draw(dzx.imag, mesh)
Draw(dzy.imag, mesh)

dz_tot = dzx * dzy
Draw(dz_tot.imag,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [15]:

Z = CF((dz_tot / (dzx**2), dz_tot / (dzy**2)))

#Z = CF((1,1))
#dz_tot = 1

fes = H1(mesh, order=4, complex=True, dirichlet='outerbnd')
u = fes.TrialFunction()
v = fes.TestFunction()
omega = 10

a = BilinearForm(fes)


inner = CF( Z[0] * grad(u)[0] * grad(v)[0] + Z[1] * grad(u)[1] * grad(v)[1])

a += inner * dx - dz_tot *omega**2*u*v * dx
a += -1j*omega*u*v * ds('outerbnd')
a.Assemble()

#f = exp(-20**2*((x+gamma-0.3)*(x+gamma-0.3)+y*gamma*y*gamma))

b = LinearForm(fes)
b += f * v * dx
b.Assemble()


gfu = GridFunction(fes)
gfu.Set(0, BND)
gfu.vec.data = a.mat.Inverse() * b.vec

Draw(gfu, mesh)#, max=0.0018, min=-0.00097)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene